<a href="https://colab.research.google.com/github/tlsgptj/emotional_BERT/blob/main/KOBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#크롤링
#기쁨, 슬픔, 놀람, 분노, 공포, 혐오
import json
import csv

with open('/content/감성대화말뭉치(최종데이터)_Training.json', 'r') as json_file:
  data = json.load(json_file)

with open('csv_file.csv', 'w', newline='', encoding='utf-8') as csv_file:
  writer = csv.writer(csv_file)
  header = data[0].keys()
  writer.writerow(header)
  for row in data:
    writer.writerow(row.values())

In [ ]:
!pip install mxnet # 코랩 환경이기 때문에 앞에 !를 붙여야 한다.
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers torch

In [ ]:
from transformers import AutoTokenizer, BertModel

# Try loading the tokenizer using AutoTokenizer
try:
    tokenizer = AutoTokenizer.from_pretrained('skt/kobert-base-v1')
except Exception as e:
    print(f"Error loading tokenizer: {e}")
    print("Please check the model name and your internet connection.")

# Load the KoBERT model
model = BertModel.from_pretrained('skt/kobert-base-v1')

# ... rest of your code ...le_with_warmup

In [ ]:
import json
import csv

# JSON 파일 로드
with open('/content/감성대화말뭉치(최종데이터)_Training.json', 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

# HS01과 HS02 추출 및 합치기
extracted_data = []
for row in data:
    talk_data = row.get('talk', {})  # 'talk' 데이터를 가져옴
    content = talk_data.get('content', {})
    hs01 = content.get('HS01', '')
    hs02 = content.get('HS02', '')

    # HS01과 HS02를 합치기
    combined_hs = f"{hs01} {hs02}".strip()  # 문자열을 합치고 불필요한 공백 제거

    if combined_hs:  # combined_hs가 비어있지 않으면 추가
        extracted_data.append([combined_hs])

# CSV 파일로 저장
csv_file_path = 'extracted_hs_data.csv'
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Combined_HS'])  # 컬럼명 작성
    writer.writerows(extracted_data)

print(f'CSV 파일이 {csv_file_path}에 저장되었습니다.')

In [ ]:
# talk에 있는 데이터만 추출해야함

In [ ]:
import pandas as pd
emotional = pd.read_csv('/content/extracted_hs_data.csv')

In [ ]:
emotional.head(20)

In [ ]:
from transformers import AutoTokenizer, BertModel

# Try loading the tokenizer using AutoTokenizer
try:
    tokenizer = AutoTokenizer.from_pretrained('skt/kobert-base-v1')
except Exception as e:
    print(f"Error loading tokenizer: {e}")
    print("Please check the model name and your internet connection.")

# Load the KoBERT model
model = BertModel.from_pretrained('skt/kobert-base-v1')

# ... rest of your code ...le_with_warmup

In [ ]:
# 필요한 과정
# 1. 텍스트 데이터 정제
# 2. 문서를 토큰으로 나누기
# 3. 불용어 제거
# 4. 모델을 적용할꺼임 KOBERT적용 등...
# 5. TF-DIF로 알고리즘 비교
# koBERT의 경우 벡터로 변환할 필요가 없다고 함 : KoBERT는 텍스트를 바로 토큰화한 뒤 임베딩 벡터로 변환하는 기능을 내장하고 있기 때문

In [ ]:
# 전처리 과정 자세히
# [영어]
# 소문자 변환
# 불필요한 공백 제거
# 구두점 제거
# 특수 문자 제거
# 숫자 제거
# 불용어 제거
# 형태소 분석
# 어간 추출
# 표제어 추출
# 철자 교정
# 단어 토큰화
# 벡터화
## -> 여기서 한국어일 경우 1번은 생략하고 분석

In [ ]:
# 구두점 제거
import string
import re
emotional['Combined_HS'] = emotional['Combined_HS'].str.translate(str.maketrans('', '', string.punctuation))
# 특수 문자 제거
emotional['Combined_HS'] = emotional['Combined_HS'].apply(lambda x: re.sub(r"[^a-zA-Z0-9가-힣\s]", "", x) if isinstance(x, str) else x)
# 숫자 제거
emotional['Combined_HS'] = emotional['Combined_HS'].apply(lambda x: re.sub(r'\d+', '', x) if isinstance(x, str) else x)

In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

# 한국어 불용어 추가
stop_words = set(stopwords.words('english'))
stop_words.update(['많아서','받아','나의','년','싶지','친구와','하네','시간이','자기','같은데','살','벌써','직장에서','결혼을','돼서','신경','조금','근데','친구들과','어떤','있지','꼭','친구에게','대해','가고','회사에','함께','돈도','나보다','이젠','빨리','친구들은','부모님께','하니','들어서','일도','아주','나랑','결혼','되었어','됐어','노후','모든','학교','할까','나서','몇','건강이','자식들이','하지만','그동안','먼저','학교에서','나와','중이야','많은','병원에','그런데','기분이야','있을','건지','나이','친한','받았어','많아', '뭘','되고','마음이','자주','점점','무슨','위해','있을까','제대로','사는','그래도','일','직장','된','친구를','아무것도','먹고','것을','모두','아무도','하나', '아빠가','참','없는데','말이야','때마다','될','어제','공부를','친구','해도','되는','전에', '하면','보면', '성적이','큰','또', '않고', '같은','것이', '없이','사람들이','얼마', '이번', '회사', '집에', '앞으로', '걸까','아직', '하지', '텐데', '매일', '부모님이', '자신이', '수가', '나에게', '생각이', '않아서', '이번에', '회사에서', '건', '그런', '그래서', '없고', '줄', '그래서', '나만', '없는', '했어', '이다', '하다', '때', '돼', '싶은데', '없어서', '날', '나이가', '같이', '난다', '것', '같아', '나는', '내가', '다', '계속', '나', '내', '더', '남편이', '아내가', '오늘', '난', '정말', '많이', '요즘', '이제', '친구가', '나도', '자꾸', '해', '좀', '나를', '했는데', '응', '그', '갑자기', '거야', '엄마가', '할수', '항상', '잘', '있는데', '어떻게 해야', '때문에', '아들이', '이제는', '친구들이', '딸이', '그렇게', '돈을', '다들', '다시', '않아', '그냥', '있어', '진짜', '돈', '너무', '안', '못', '계', '수', '하고', '없어', '할', '어떻게', '봐', '게', '하는', '우리', '왜', '거', '해서', '한', '있는', '들어', '싶어', '해야', '같아서', '하는데', '일을', '다른', '할지', '걸', '이', '이렇게', '일이', '말을', '것도', '몸이', '나한테', '지금', '있어서', '돈이', '보니', '이런', '사람이'])

def remove_stopwords(text):
  if isinstance(text, str):
    tokens = text.split()
    result = [word for word in tokens if not word in stop_words]
    return " ".join(result)
  else:
    return text

emotional['Combined_HS'] = emotional['Combined_HS'].apply(remove_stopwords)

In [ ]:
!apt-get update
!apt-get install -y fonts-nanum

In [ ]:
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Combine all text in the 'Combined_HS' column into a single string
text = ' '.join(emotional['Combined_HS'].astype(str).tolist())

# Generate the word cloud
wordcloud = WordCloud(width=800, height=800, background_color='white', font_path='/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf').generate(text)

# Display the word cloud
plt.figure(figsize=(8, 8), facecolor=None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()

In [ ]:
emotional.head(20)

In [ ]:
import csv
import re
from konlpy.tag import Okt
okt = Okt()

emotion_words = {
    "기쁨": ["지","만족", "자랑", "어", "좋", "좋아", "기분", "다행", "원하", "웃", "성공", "합격", "가벼워", "행복", "편해", "은", "하는", "기쁘", "다"],
    "슬픔": ["억울","해","속상","슬프","어","눈물이","아파","해", "속상","해", "받", "상처", "원망", "스러워", "안쓰럽", "낙담", "슬프", "힘들", "미안", "후회", "억울", "괴로워", "외롭", "한심", "눈물", "섭섭", "취소", "버리", "망하", "실망", "싸우", "불쌍", "서운", "미워", "서글프", "슬프", "씁쓸", "서러"],
    "걱정": ["무서워","불안","해","당황", "혼란", "두렵", "모르", "초조", "걱정", "어떡하", "불안", "고민", "마비", "하다"],
    "분노": ["가","역겨", "환멸", "원망", "얄밉", "짜증", "화", "스트레스", "답답", "분하", "배신", "버리", "의심"],
    "공포": ["모르겠어","무섭", "두렵", "두려움", "무서움", "힘들"],
    "우울": ["힘들어","힘든","힘드네","힘들", "외롭", "어렵", "버겁", "우울","해", "고독", "쓸쓸", "하네", "하다"],
    "부끄러움": ["당황", "창피", "부끄럽", "실수", "당혹", "민망", "망신"],
    "감사": ["감사"],
    "놀람": ["놀랐"],
    "후회": ["후회", "돼", "스러워"],
    "기타": []
}



# 감정 레이블
emotion_labels = {
    "기쁨": 0,
    "슬픔": 1,
    "걱정": 2,
    "분노": 3,
    "공포": 4,
    "우울": 5,
    "부끄러움": 6,
    "감사":7,
    "놀람":8,
    "후회":9,
    "기타":10
}

# 텍스트에서 문장 부호 제거 함수
def clean_text(text):
    # 정규 표현식을 사용하여 문자와 숫자가 아닌 것을 공백으로 치환
    text = re.sub(r'[^\w\s]', '', text)
    return text

# 텍스트에서 감정 레이블 추출 함수 (형태소 분석 추가)
def label_emotion(text):
    text = clean_text(text)  # 문장 부호 제거
    tokens = okt.morphs(text)  # 형태소 분석을 통해 텍스트를 토큰화
    emotion_count = {emotion: 0 for emotion in emotion_words}  # 감정별 카운트를 위한 딕셔너리

    for emotion, words in emotion_words.items():
        for word in words:
            if word in tokens:
                emotion_count[emotion] += 1  # 감정 단어가 발견되면 카운트 증가

    max_emotion = max(emotion_count, key=emotion_count.get)  # 가장 많이 등장한 감정 선택
    if emotion_count[max_emotion] > 0:
        return emotion_labels[max_emotion]
    return emotion_labels["기타"]  # 감정 단어가 없을 경우 "기타"로 처리

# CSV 파일에서 텍스트 읽고 라벨링한 후 저장[[/0]]
def label_csv(input_file, output_file):
    with open(input_file, mode='r', encoding='utf-8') as infile, open(output_file, mode='w', newline='', encoding='utf-8') as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile)

        # 헤더 작성
        header = next(reader)  # 기존 CSV의 헤더를 가져옴
        header.append('Emotion_Label')  # 새로운 열 추가
        writer.writerow(header)

        # 각 행에 대해 감정 레이블 추가
        for row in reader:
            text = row[0]  # 텍스트가 들어 있는 열 (필요 시 인덱스 조정)
            emotion_label = label_emotion(text)
            row.append(emotion_label)
            writer.writerow(row)

# 10번 라벨(기타)만 추출하는 함수
def extract_label_10(input_file, output_file):
    with open(input_file, mode='r', encoding='utf-8') as infile, open(output_file, mode='w', newline='', encoding='utf-8') as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile)

        # 헤더 작성
        header = next(reader)
        writer.writerow(header)

        # 10번 레이블(기타)인 행만 추출하여 작성
        for row in reader:
            if row[-1] == '10':  # 마지막 열이 '10'(기타 레이블)인 경우
                writer.writerow(row)

# 10번 라벨(기타)만 추출하고 개수 카운트하는 함수
def extract_label_10(input_file, output_file):
    count = 0  # 10번 라벨의 개수를 세기 위한 변수
    with open(input_file, mode='r', encoding='utf-8') as infile, open(output_file, mode='w', newline='', encoding='utf-8') as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile)

        # 헤더 작성
        header = next(reader)
        writer.writerow(header)

        # 10번 레이블(기타)인 행만 추출하여 작성
        for row in reader:
            if row[-1] == '10':  # 마지막 열이 '10'(기타 레이블)인 경우
                writer.writerow(row)
                count += 1  # 10번 레이블 개수 증가

    print(f"10번 라벨(기타) 데이터는 총 {count}개 있습니다.")
    return count

# 라벨 10번만 추출하여 저장할 파일 경로
label_10_output = 'label_10_output.csv'
count_10 = extract_label_10(output_csv, label_10_output)

print(f"라벨 10번 데이터가 '{label_10_output}'에 저장되었으며, 총 {count_10}개의 행이 10번 라벨로 분류되었습니다.")

# CSV 파일 경로 설정
input_csv = '/content/extracted_hs_data.csv'  # 감정 분석할 CSV 파일 경로
output_csv = 'labeled_output.csv'  # 라벨링된 결과를 저장할 CSV 파일 경로

# CSV 파일에 라벨링 적용
label_csv(input_csv, output_csv)


In [ ]:
!apt-get update
!apt-get install -y fonts-nanum

In [ ]:
import pandas as pd
emotion = pd.read_csv('/content/label_10_output.csv')

In [ ]:
emotion.tail(20)

In [ ]:
# 구두점 제거
import string
import re
emotion['Combined_HS'] = emotion['Combined_HS'].str.translate(str.maketrans('', '', string.punctuation))
# 특수 문자 제거
emotion['Combined_HS'] = emotion['Combined_HS'].apply(lambda x: re.sub(r"[^a-zA-Z0-9가-힣\s]", "", x) if isinstance(x, str) else x)
# 숫자 제거
emotion['Combined_HS'] = emotion['Combined_HS'].apply(lambda x: re.sub(r'\d+', '', x) if isinstance(x, str) else x)

In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

# 한국어 불용어 추가
stop_words = set(stopwords.words('english'))
stop_words.update(['아무래도','그래','있었어','어려워','기분이','느껴져','고민이야','친구는','있었는데','좋겠어','죄책감이','싶은','보고','싫어','맞아','혼자','열심히','많아서','받아','나의','년','싶지','친구와','하네','시간이','자기','같은데','살','벌써','직장에서','결혼을','돼서','신경','조금','근데','친구들과','어떤','있지','꼭','친구에게','대해','가고','회사에','함께','돈도','나보다','이젠','빨리','친구들은','부모님께','하니','들어서','일도','아주','나랑','결혼','되었어','됐어','노후','모든','학교','할까','나서','몇','건강이','자식들이','하지만','그동안','먼저','학교에서','나와','중이야','많은','병원에','그런데','기분이야','있을','건지','나이','친한','받았어','많아', '뭘','되고','마음이','자주','점점','무슨','위해','있을까','제대로','사는','그래도','일','직장','된','친구를','아무것도','먹고','것을','모두','아무도','하나', '아빠가','참','없는데','말이야','때마다','될','어제','공부를','친구','해도','되는','전에', '하면','보면', '성적이','큰','또', '않고', '같은','것이', '없이','사람들이','얼마', '이번', '회사', '집에', '앞으로', '걸까','아직', '하지', '텐데', '매일', '부모님이', '자신이', '수가', '나에게', '생각이', '않아서', '이번에', '회사에서', '건', '그런', '그래서', '없고', '줄', '그래서', '나만', '없는', '했어', '이다', '하다', '때', '돼', '싶은데', '없어서', '날', '나이가', '같이', '난다', '것', '같아', '나는', '내가', '다', '계속', '나', '내', '더', '남편이', '아내가', '오늘', '난', '정말', '많이', '요즘', '이제', '친구가', '나도', '자꾸', '해', '좀', '나를', '했는데', '응', '그', '갑자기', '거야', '엄마가', '할수', '항상', '잘', '있는데', '어떻게 해야', '때문에', '아들이', '이제는', '친구들이', '딸이', '그렇게', '돈을', '다들', '다시', '않아', '그냥', '있어', '진짜', '돈', '너무', '안', '못', '계', '수', '하고', '없어', '할', '어떻게', '봐', '게', '하는', '우리', '왜', '거', '해서', '한', '있는', '들어', '싶어', '해야', '같아서', '하는데', '일을', '다른', '할지', '걸', '이', '이렇게', '일이', '말을', '것도', '몸이', '나한테', '지금', '있어서', '돈이', '보니', '이런', '사람이'])

def remove_stopwords(text):
  if isinstance(text, str):
    tokens = text.split()
    result = [word for word in tokens if not word in stop_words]
    return " ".join(result)
  else:
    return text

emotion['Combined_HS'] = emotion['Combined_HS'].apply(remove_stopwords)

In [ ]:
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Combine all text in the 'Combined_HS' column into a single string
text = ' '.join(emotion['Combined_HS'].astype(str).tolist())

# Generate the word cloud
wordcloud = WordCloud(width=800, height=800, background_color='white', font_path='/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf').generate(text)

# Display the word cloud
plt.figure(figsize=(8, 8), facecolor=None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()

In [ ]:
import pandas as pd
df1 = pd.read_csv('/content/labeled_output.csv')

In [ ]:
df1.tail(50)

In [ ]:
# 모든 데이터를 3으로 라벨링을 해버려서 kobert는 파인 튜닝이 필요할 것 같음

In [ ]:
nan_data = df1[df1['Emotion_Label'].isnull()] # Use df1 instead of emotional

In [ ]:
nan_data = df1[df1['Emotion_Label'].isnull()]
num_nan_data = len(nan_data)

print(f"nan_data는 {num_nan_data}개 있습니다.")

In [ ]:
!apt-get update
!apt-get install -y fonts-nanum

In [ ]:
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Combine all text in the 'Combined_HS' column into a single string
text = ' '.join(nan_data['Combined_HS'].astype(str).tolist())

# Generate the word cloud
wordcloud = WordCloud(width=800, height=800, background_color='white', font_path='/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf').generate(text)

# Display the word cloud
plt.figure(figsize=(8, 8), facecolor=None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()

In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

# 한국어 불용어 추가
stop_words = set(stopwords.words('english'))
stop_words.update(['맞아','않아','거야','없어','했어','혼자','응','싶어','있어','해','같아','많아서','받아','나의','년','싶지','친구와','하네','시간이','자기','같은데','살','벌써','직장에서','결혼을','돼서','신경','조금','근데','친구들과','어떤','있지','꼭','친구에게','대해','가고','회사에','함께','돈도','나보다','이젠','빨리','친구들은','부모님께','하니','들어서','일도','아주','나랑','결혼','되었어','됐어','노후','모든','학교','할까','나서','몇','건강이','자식들이','하지만','그동안','먼저','학교에서','나와','중이야','많은','병원에','그런데','기분이야','있을','건지','나이','친한','받았어','많아', '뭘','되고','마음이','자주','점점','무슨','위해','있을까','제대로','사는','그래도','일','직장','된','친구를','아무것도','먹고','것을','모두','아무도','하나', '아빠가','참','없는데','말이야','때마다','될','어제','공부를','친구','해도','되는','전에', '하면','보면', '성적이','큰','또', '않고', '같은','것이', '없이','사람들이','얼마', '이번', '회사', '집에', '앞으로', '걸까','아직', '하지', '텐데', '매일', '부모님이', '자신이', '수가', '나에게', '생각이', '않아서', '이번에', '회사에서', '건', '그런', '그래서', '없고', '줄', '그래서', '나만', '없는', '했어', '이다', '하다', '때', '돼', '싶은데', '없어서', '날', '나이가', '같이', '난다', '것', '같아', '나는', '내가', '다', '계속', '나', '내', '더', '남편이', '아내가', '오늘', '난', '정말', '많이', '요즘', '이제', '친구가', '나도', '자꾸', '해', '좀', '나를', '했는데', '응', '그', '갑자기', '거야', '엄마가', '할수', '항상', '잘', '있는데', '어떻게 해야', '때문에', '아들이', '이제는', '친구들이', '딸이', '그렇게', '돈을', '다들', '다시', '않아', '그냥', '있어', '진짜', '돈', '너무', '안', '못', '계', '수', '하고', '없어', '할', '어떻게', '봐', '게', '하는', '우리', '왜', '거', '해서', '한', '있는', '들어', '싶어', '해야', '같아서', '하는데', '일을', '다른', '할지', '걸', '이', '이렇게', '일이', '말을', '것도', '몸이', '나한테', '지금', '있어서', '돈이', '보니', '이런', '사람이'])

def remove_stopwords(text):
  if isinstance(text, str):
    tokens = text.split()
    result = [word for word in tokens if not word in stop_words]
    return " ".join(result)
  else:
    return text

nan_data['Combined_HS'] = nan_data['Combined_HS'].apply(remove_stopwords)

In [ ]:
pip install konlpy

In [ ]:
from konlpy.tag import Okt
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Okt 형태소 분석기 초기화
okt = Okt()

# 한국어 불용어 리스트
stop_words = set(['이야기','가족','아이','자식','집','이야','사람','연락','공부','말','애','취업','준비','남편','라고','얘기','로','에서','말','인데','으로','의','까지','생각','들','가','생각','는','도','를','만','에','을','은','맞아','않아','거야','없어','했어','혼자','응','싶어','있어','해','같아','많아서','받아','나의','년','싶지','친구와','하네','시간이','자기','같은데','살','벌써','직장에서','결혼을','돼서','신경','조금','근데','친구들과','어떤','있지','꼭','친구에게','대해','가고','회사에','함께','돈도','나보다','이젠','빨리','친구들은','부모님께','하니','들어서','일도','아주','나랑','결혼','되었어','됐어','노후','모든','학교','할까','나서','몇','건강이','자식들이','하지만','그동안','먼저','학교에서','나와','중이야','많은','병원에','그런데','기분이야','있을','건지','나이','친한','받았어','많아', '뭘','되고','마음이','자주','점점','무슨','위해','있을까','제대로','사는','그래도','일','직장','된','친구를','아무것도','먹고','것을','모두','아무도','하나', '아빠가','참','없는데','말이야','때마다','될','어제','공부를','친구','해도','되는','전에', '하면','보면', '성적이','큰','또', '않고', '같은','것이', '없이','사람들이','얼마', '이번', '회사', '집에', '앞으로', '걸까','아직', '하지', '텐데', '매일', '부모님이', '자신이', '수가', '나에게', '생각이', '않아서', '이번에', '회사에서', '건', '그런', '그래서', '없고', '줄', '그래서', '나만', '없는', '했어', '이다', '하다', '때', '돼', '싶은데', '없어서', '날', '나이가', '같이', '난다', '것', '같아', '나는', '내가', '다', '계속', '나', '내', '더', '남편이', '아내가', '오늘', '난', '정말', '많이', '요즘', '이제', '친구가', '나도', '자꾸', '해', '좀', '나를', '했는데', '응', '그', '갑자기', '거야', '엄마가', '할수', '항상', '잘', '있는데', '어떻게 해야', '때문에', '아들이', '이제는', '친구들이', '딸이', '그렇게', '돈을', '다들', '다시', '않아', '그냥', '있어', '진짜', '돈', '너무', '안', '못', '계', '수', '하고', '없어', '할', '어떻게', '봐', '게', '하는', '우리', '왜', '거', '해서', '한', '있는', '들어', '싶어', '해야', '같아서', '하는데', '일을', '다른', '할지', '걸', '이', '이렇게', '일이', '말을', '것도', '몸이', '나한테', '지금', '있어서', '돈이', '보니', '이런', '사람이'])

# 불용어 제거 함수 (Okt 형태소 분석 사용)
def remove_stopwords(text):
    if isinstance(text, str):
        tokens = okt.morphs(text)  # 형태소 분석을 사용한 토큰화
        result = [word for word in tokens if word not in stop_words]  # 불용어 제거
        return " ".join(result)
    else:
        return text

# 데이터셋에서 Combined_HS 칼럼 불용어 제거 적용
nan_data['Combined_HS'] = nan_data['Combined_HS'].apply(remove_stopwords)

# 워드 클라우드 생성
text = ' '.join(nan_data['Combined_HS'].astype(str).tolist())
wordcloud = WordCloud(width=800, height=800, background_color='white', font_path='/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf').generate(text)

# 워드 클라우드 표시
plt.figure(figsize=(8, 8), facecolor=None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()
